# Introduction

<font color = green> <strong>MUST READ:</strong></font> This Notebook tutorial will allow one to create temporary key files for all applicable slots and program into the device. These keys would be used for various operations performed by various usecase Notebooks and Embedded projects provided in the DesignSuite.

<font color = green>These keys are generated using python modules and is stored in computer's hard disk, this is not secure and is not recommended for actual production. In production environment these keys need to be stored in a secure storage.</font>

Before running this Notebook,
1. Make sure CryptoAuth Trust Platform is having factory reset program

### Prerequisites:
This step of the tutorial will attempt to load all the necessary modules and their dependencies on your machine. If the modules are already installed you can safely step over the next Tutorial step.

<font color = orange> <strong>Note</strong></font> 
1. Installation time for prerequisites depends upon system and network speed.
2. Installing prerequisites for the first time takes more time and watch the kernel status for progress.Following image helps to locate the Kernel status,
<center><img src="../../assets/notebook/img/kerner_status.png" alt="**Check Kernel Status**" /></center>


3. Installing prerequisites gives the following error and it can be safely ignored. Functionality remains unaffected.
    - <font color = orange> azure-cli 2.0.76 has requirement colorama~=0.4.1, but you'll have colorama 0.3.9 which is incompatible.</font>
    - <font color = orange>  azure-cli 2.0.76 has requirement pytz==2019.1, but you'll have pytz 2019.3 which is incompatible. </font>

In [1]:
import sys, os

home_path = os.path.dirname(os.path.dirname(os.path.realpath(os.getcwd())))
module_path = os.path.join(home_path, 'assets', 'python')
if not module_path in sys.path:
    sys.path.append(module_path)

from requirements_helper import requirements_installer
obj = requirements_installer(os.path.join(home_path, 'assets', 'requirements.txt'))

Checking module (1 of 17): awscli
-----------------------------------------
Checking module (2 of 17): cryptoauthlib
-----------------------------------------
Checking module (3 of 17): cryptography
-----------------------------------------
Checking module (4 of 17): ipywidgets
-----------------------------------------
Checking module (5 of 17): pyasn1-modules
-----------------------------------------
Checking module (6 of 17): asn1crypto
-----------------------------------------
Checking module (7 of 17): pytz
-----------------------------------------
Checking module (8 of 17): python-jose
-----------------------------------------
Checking module (9 of 17): pyOpenSSL
-----------------------------------------
Checking module (10 of 17): boto3
-----------------------------------------
Checking module (11 of 17): requests-aws4auth
-----------------------------------------
Checking module (12 of 17): hexrec
-----------------------------------------
Checking module (13 of 17): hidapi
-----

### Setup Modules and Hardware
This step loads the required modules and helper functions to perform the resource generation sequence. It also 
1. Initializes the interface with TrustFLEX hardware and establishes commmunication with TrustFLEX.
2. Performs device initialization to verify the actual device attached

In [2]:
####################################### Load Modules
import os, fnmatch
import cryptography
import ipywidgets as widgets
import warnings

from helper import *
from cryptoauthlib import *
from certs_handler import *
from trustplatform import *

warnings.filterwarnings('ignore')
print("Importing modules - Successful")

#TFLXTLS device I2C address 0x6C; #TNGTLS device I2C address 0x6A; 
common_helper.connect_to_secure_element('ATECC608A', ATCAKitType.ATCA_KIT_I2C_IFACE, 0x6C)
print("Device initialization - Successful")

Importing modules - Successful
Device initialization - Successful


### Generate Keys
This step of Notebook creates key files for all applicable slots and programs  into the device.

<font color=green>The keys are generated here using python module and is stored in computer's hard disk, this is not secure and is not recommended for actual production. In actual production environment these keys need to be stored in a secure storage as depitected below.</font>

NOTE: While writing symmetric key into secure element it has to be encrypted with IO protection key. So here, Slot 6 (IO protection key) is written before slot 5 (Symmetric key).

In [3]:
print('\n\r---------------------------------------------')
slots = [0,1,2,3,4,6,5,7,8,9,10,11,12,13,14,15] 
for i in slots:
    status = create_resource_and_program(i)
    if(status != True):
        raise ValueError("Resource gererator failure")

# Additional step to generate files for C-examples (ignore for Notebooks)
create_resources_for_c_examples();
print('\n\r---------------------------------------------')
print('Key generation - Success')
print('---------------------------------------------\n\r')


---------------------------------------------
Slot 0 is a private key slot, no action required
Slot 1 is a private key slot, no action required
Slot 2 is a private key slot, no action required
Slot 3 is a private key slot, no action required
Slot 4 is a private key slot, no action required
Slot 6 is a secret key, created slot_6_secret_key.pem and programmed

NOTE: While writing symmetric key into secure element it has to be encrypted with IO protection key. So here, Slot 6 (IO protection key) is written before slot 5 (Symmetric key) 

Slot 5 is a secret key, created slot_5_secret_key.pem and programmed
Slot 7 is a secureboot digest slot, slot can only be written through secureboot command
Slot 8 is a general purpose slot of size 416 bytes, no action required
Slot 9 is a secret key, created slot_9_secret_key.pem and programmed
Slot 10 is a certificate slot, no action required now, will be updated as part of Generate Certificates
Slot 11 is a certificate slot, no action required now, wi

### Generate Ceritificates
This step of the Notebook generates certificates and Manifest files for the connected devices. The certificate can be MCHP standard certificate or the custom certificate in which Organization name can be User specific. 

This generates root CA, Signer CA as per the MCHP standard template. Manifests generated here will be signed by the log signer created in this step, Whereas Manifests downloaded from Microchip Direct will be signed by Microchip.

<font color=green>The certificates are generated using python module and CA's private keys are stored in computer's hard disk, this is not secure and is not recommended for actual production. In actual production environment these keys need to be stored in a secure storage</font>

In [5]:
def generate_mchp_cert(b):
    print('--------------------------------------------')
    print('MCHP Certs processing...')
    mchp_cert()
    print('MCHP Certificate processing completed successfully')
    print('--------------------------------------------')
def get_org_name(b):
    text = widgets.Text(value='', description='Org Name:', style={'description_width': '150px'})
    text.on_submit(generate_cust_cert)
    display(text)
def generate_cust_cert(b):
    print('Custom Certs processing...')
    custom_cert(b.value)
    print('--------------------------------------------')
tooltip = 'Attemps to restore MCHP certs'
mchp_cert_button = widgets.Button(description = "MCHP Cert", tooltip=tooltip, layout=widgets.Layout(width='auto'))
mchp_cert_button.on_click(generate_mchp_cert)
tooltip = 'Attemps to load Custom certs'
cust_cert_button = widgets.Button(description = "Custom Cert", tooltip=tooltip, layout=widgets.Layout(width='auto'))
cust_cert_button.on_click(get_org_name)

certs_handler.create_manifest_log_signer()
print('\n\nSelect the Certificate Type to prototype')
display(widgets.HBox((mchp_cert_button, cust_cert_button)))

------------- Creating Manifest Log Signer ----------- 
Loading Manifest logger key
Generating self-signed logging certificate
    Saving to log_signer.crt
------------------------------------------------------


Select the Certificate Type to prototype


Text(value='', description='Org Name:', style=DescriptionStyle(description_width='150px'))

Custom Certs processing...
Device contains custom device and signer certificates
Adjusted Orgname for size and spaces:test_org2_______________
Read device serial number...OK (SN: 0123BCE5492A5B4701)
Read device public key from slot 0...OK (Public Key: B3102C59A392087B27DFE7708DBA46864120D8CD7D7F764AF42A603DC204F9C684A60021EBA46C32911B96D06BE28C4DEC4ABB52E4078951CCA34D1C27D0BED2)
Generating device certificate...OK (saved to device_0123BCE5492A5B4701.crt)
Saving signer certificate to device...OK
Saving device certificate to device...OK
Thing ID 655487aaf0ef29068f90808ecf84abfd7cff213c

---------------------------------------------
Custom certificate generation and provisioning - SUCCESS
---------------------------------------------

Validate root certificate...OK
-----BEGIN CERTIFICATE-----
MIIB8TCCAZegAwIBAgIQV6Xd9dhCIL7lVTCY0iHk3jAKBggqhkjOPQQDAjBPMSEw
HwYDVQQKDBh0ZXN0X29yZzJfX19fX19fX19fX19fX18xKjAoBgNVBAMMIUNyeXB0
byBBdXRoZW50aWNhdGlvbiBSb290IENBIDAwMjAgFw0yMDAzMTAxMzEyNTRaGA8y
MDYwM

Keys are generated and stored in the device, the same key will be available in the the same directory as .pem files. For certificates the tool will generate keys, certificates and associated .c and .h files. 
    
<font color=green> This text marks the end of this tool, after this you can run the usecase examples.